So it's not possible to get the method ID during a token transfer and while I can get it from the standard tx list, the token transfers are not part of that list.

The easiest way is probable just to take the transfers from the Sickle contract to my wallet and check to make sure no value is above a certain amount, like $5k or something, which will be an exit and not rebalance dust or harvesting.  

To do this right, I would need to get the tx hashes and then look them up and figure out what the method is for each of them.

In [1]:
import transaction_class as tc
# Method IDs
rebalance_for = '0xe6fb317f'
harvest = '0x5ec5999e'
wallet = '0x8F9ACea6bC63470AA83169637CBf132482A28d55'
sender = '0x58C8bc977f1C53d5Dc085aE26732fdaFe675646d'
start_block = 25707612

wal = tc.Transactions(wallet)
wal.get_transactions(start_block)
list = wal.filter_tx_list_by_sender(sender)

Received transactions for USDC
Received transactions for WETH
Received transactions for cbBTC
Received transactions for AERO


In [2]:
import pandas as pd
df = pd.DataFrame(list)
df['timeStamp'] = pd.to_datetime(df['timeStamp'].astype(int), unit='s')
df['value'] = df['value'].astype(float) / 10 ** df['tokenDecimal'].astype(int)
# df.to_csv('wallet_tx.csv')

In [ ]:
import requests
from datetime import datetime, timedelta

def get_historical_token_price(token_address, transaction_timestamp):
    base_url = 'https://api.coingecko.com/api/v3/coins/markets'
    
    # Convert timestamp to date in 'YYYY-MM-DD' format
    transaction_date = datetime.fromtimestamp(transaction_timestamp).date()

    params = {
        'vs_currency': 'usd',
        'ids': token_address,
        'order': 'market_cap_desc',
        'per_page': '1',
        'page': '1',
        'date': transaction_date.strftime('%Y-%m-%d')
    }

    response = requests.get(base_url, params=params)
    data = json.loads(response.text)

    if data:
        price = data[0]['price']
    else:
        price = 0

    return price


In [ ]:
def track_transactions(wallet_address, start_block, method_ids=None):
    transactions = get_transactions(wallet_address, start_block, method_ids)

    grouped_transactions = {}

    for transaction in transactions:
        token_address = transaction['address']
        amount = int(transaction['data'].split(',')[0], 16)
        method_id = transaction['topics'][1]

        if token_address not in grouped_transactions:
            grouped_transactions[token_address] = {
                'total_amount': amount,
                'usd_value': 0,
                'transactions': [],
            }
        else:
            grouped_transactions[token_address]['total_amount'] += amount

        grouped_transactions[token_address]['transactions'].append({
            'amount': amount,
            'block': transaction['blockNumber'],
            'timestamp': transaction['timeStamp'],
            'method_id': method_id,
        })

    for token_address, data in grouped_transactions.items():
        token_price = get_token_price(token_address)
        data['usd_value'] = data['total_amount'] * token_price

    return grouped_transactions

# Example usage:
wallet_address = "0xYourWalletAddress"  # Replace with your actual wallet address
start_block = 1234567  # Replace with the block number from where you want to start tracking
method_ids = [  # Replace with the method IDs you want to filter by
    "0xYourMethodID1",
    "0xYourMethodID2",
]

transaction_data = track_transactions(wallet_address, start_block, method_ids)

for token_address, transactions in transaction_data.items():
    print(f"Token: {token_address}")
    print(f"Total amount: {transactions['total_amount']}")
    print(f"USD value: {transactions['usd_value']}")
    print(f"Transactions:")
    for tx in transactions['transactions']:
        print(f"  - Amount: {tx['amount']}")
        print(f"    Block: {tx['block']}")
        print(f"    Timestamp: {tx['timestamp']}")
        print(f"    Method ID: {tx['method_id']}")
        print("------------------")
